In [40]:
from skimage import data, draw,color, data, restoration
from skimage import transform, util
from scipy import misc, fftpack, signal, ndimage
import numpy as np
from numpy.linalg import inv
from skimage import filters, color
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageEnhance
import matplotlib.animation as animation
from scipy.misc import imresize
from scipy import ndimage as nd
import scipy

In [41]:
scale = 8
rgb =  plt.imread('casa.jpg')
rgb = imresize(rgb, ( rgb.shape[0]/scale,rgb.shape[1]/scale,3),interp='bilinear')
img_LAB = color.rgb2lab(rgb.astype('float'))
img_gris = color.rgb2gray(rgb)
print img_gris.shape
print rgb.shape
#fig = plt.figure(121) 
#plt.imshow(rgb)
#plt.title('Original Image')
#fig = plt.figure(122) 
#plt.imshow(img_LAB[:,:,2])
#plt.title('Gray image')
#plt.show()

(60, 80)
(60, 80, 3)


In [42]:
def gradient(img):
    img = color.rgb2gray(img)
    gx, gy = np.gradient(img)
    grad = np.sqrt(gx*gx + gy*gy)
    return grad
grad = gradient(img_gris)
#normalitzacio d'imatge
def normalitzar(imatge):
    return np.real((imatge - np.min(imatge)) * 256 / (np.max(imatge) - np.min(imatge)))

In [43]:
#normalitzo les imatges a 255
rgb=normalitzar(rgb)
img_gris=normalitzar(img_gris)
img_grad=gradient(rgb)
img_grad=normalitzar(img_grad)
print img_grad.shape
#plt.imshow(rgb.astype('uint8'))
#plt.title('Original Image')
#fig = plt.figure(122) 
#plt.imshow(img_gris.astype('uint8'),cmap="gray")
#plt.title('Gray image')
#fig = plt.figure(123) 
#plt.imshow(img_grad.astype('uint8'),cmap="gray")
#plt.title('Contorn')
#plt.show()

(60, 80)


In [44]:
class get_mouse_click():
    
    #Mouse interaction interface for radial distortion removal.
    def __init__(self, img):
      height, width = img.shape[:2]
      self.figure = plt.imshow(img, extent=(0, width, height, 0))
      plt.gray()
      plt.title('select the object to remove')
      plt.xlabel('Select sets of  points with left mouse button,\n'
                 'click right button to close the polygon.')
      plt.connect('button_press_event', self.button_press)
      plt.connect('motion_notify_event', self.mouse_move)

      self.img = np.atleast_3d(img)
      self.points = []
      self.centre = np.array([(width - 1)/2., (height - 1)/2.])

      self.height = height
      self.width = width

      self.make_cursorline()
      self.figure.axes.set_autoscale_on(False)

      plt.show()
      plt.close()

    def make_cursorline(self):
        self.cursorline, = plt.plot([0],[0],'r:+', linewidth=2,markersize=15,markeredgecolor='b')

    def button_press(self,event):
        # Register mouse clicks.
        if (event.button == 1 and event.xdata and event.ydata):
            self.points.append((event.xdata,event.ydata))
            plt.gca().lines.append(self.cursorline)
            plt.gca()
            self.make_cursorline()
        if (event.button != 1):
            self.points.append((self.points[0][0],self.points[0][1]))
            plt.close()
            return self.points

    def mouse_move(self,event):
        # Handle cursor drawing.
        pts_last_set=len(self.points)
        pts = np.zeros((pts_last_set+1,2))
        if pts_last_set > 0:
            # Line follows up to 3 clicked points:
            pts[:pts_last_set] = self.points[-pts_last_set:]
            # The last point of the line follows the mouse cursor
        pts[pts_last_set:] = [event.xdata,event.ydata]
        self.cursorline.set_data(pts[:,0], pts[:,1])
        plt.draw()
def compute_mask(width,height,polygon):
    
    img = Image.new('L', (width, height), 0)
    ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
    mask = np.array(img)
    #mask100 = 100*np.ones([mask.shape[0],mask.shape[1]])
    #mask=mask*101
    #mask = 1. - mask; # switch 0s and 1s
    return mask
def convine_mask(zona_delete,img_gris):
    #si tenim 1 a la macara tenim forat
    for i in xrange(len(img_gris)):
        for j in xrange(len(img_gris[0])):
            if zona_delete[i][j]==1:
                img_gris[i][j]=-100
    return img_gris
def buscar_pix(img,t,r,finestra):   
    for i in xrange(len(img)):
        for j in xrange(len(img[0])):
            
            if img[i][j]!=-100:
                #if img[i][j]-img[t][r]<=0.1 and grad[i][j]-grad[t][r]<=0.1:
                if img[i][j]-img[t][r]<=0.8:
                    pix=(img[i][j]+img[i-1][j-1]+img[i-1][j]+img[i][j-1])/4
                    #print "t,r",img_mask[t][r]
                    #print "i,j",img_mask[i][j]
                    return pix
def reccorrer_img(finestra,finestra_validar,img_gris,t,r,vent):
    dif=19999999990.
    #valors=[]
    for i in xrange(len(img_gris)-vent):
        for j in xrange(len(img_gris[0])-vent):
            #valor = millors_pix(finestra,finestra_validar,img_gris,i,j)
            #si la puntuació es inferior al valor es que es prou semblant
            valor = millors_pix(finestra,finestra_validar,img_gris,i,j)
            
            if valor < dif:
                print valor, dif, i, j
                valors = i,j
                dif = valor
    print "i i j",i,j
    return valors
                
def millors_pix(finestra,finestra_validar,img_gris,t,r):
    #print finestra.shape
    #print finestra_validar.shape
    total_ones_mask = finestra_validar.sum()
    #aqui fariem el calcul de la formula explicada pel professor
    #1/numero de 1 de la mascara*(Ixy-Fxy)² ssd = float(np.sum((img_gris[:,:,0:3] - finestra[:,:,0:3])**2))
    for i in xrange(len(finestra_validar)):
        for j in xrange(len(finestra_validar[0])):
            ssd = float(np.sum(finestra_validar[i][j]*((img_gris[t+i,r+j] - finestra[i][j])**2)))

    #print finestra_validar
    #print (1./total_ones_mask)*ssd
    return (1./total_ones_mask)*ssd

In [45]:
rdi = get_mouse_click(img_gris.astype('uint8'))
height,width=img_gris.shape
zona_delete = compute_mask(width,height,rdi.points)
##################3faltaria el tema que va comentar d'usuar una imatge copia xk no afectes.
#####################per altra part a la herba no funciona
#calculo el primer contorn a substituir restant la mascara menys ella mateixa erosionada
cont=0
vent =4
for p in xrange(5):
    zona_delete_erosio=ndimage.binary_erosion(zona_delete)
    #busco el contorn mes extern de la part a substituir
    delete_ceva =zona_delete - zona_delete_erosio 
    #actualitzo la mascara amb el que ja haurem substituit
    
    finestra = np.zeros((vent,vent))
    finestra_validar = np.ones((vent,vent), dtype=np.int)
    img_gris_convine=convine_mask(zona_delete,img_gris)
    zona_delete = zona_delete_erosio
    for i in xrange(len(delete_ceva)):
        for j in xrange(len(delete_ceva[0])):
            if delete_ceva[i][j]==1:
                #print i,j
                finestra=zona_delete[i-vent/2:i+vent/2,j-vent/2:j+vent/2]
                #finestra2 per multiplicar on tenim 0 on hi haura el forat
                finestra_validar=abs(finestra-finestra_validar)
                #finestra_retorna=funcio que retornara la finestra mes similar
                t,r=reccorrer_img(finestra,finestra_validar,img_gris_convine,i,j,vent)
                img_gris[i][j]=img_gris[t][r]
                misc.imsave('imatges/'+str(cont+1)+'.png',img_gris)
                cont= cont +1
                #fig = plt.figure(123) 
                #plt.imshow(img_gris)
                #plt.title('Contorn')
                #plt.show()
            


4056.23493217 19999999990.0 0 0
4053.86739614 4056.23493217 0 1
4046.50576361 4053.86739614 0 2
4039.15082134 4046.50576361 0 3
2435.06321834 4039.15082134 1 0
1109.46661306 2435.06321834 2 0
1006.91139001 1109.46661306 3 1
760.059299047 1006.91139001 6 0
700.317771561 760.059299047 7 0
672.736223068 700.317771561 7 1
635.87848223 672.736223068 7 3
386.083138847 635.87848223 17 46
253.451199927 386.083138847 17 47
235.098026117 253.451199927 18 45
213.010456754 235.098026117 18 46
199.512418787 213.010456754 19 44
192.687949605 199.512418787 22 64
187.391667199 192.687949605 23 67
180.986764822 187.391667199 31 36
115.184829079 180.986764822 31 37
99.0431055008 115.184829079 32 27
76.5578765554 99.0431055008 32 28
76.2329151524 76.5578765554 32 29
69.0522012554 76.2329151524 32 37
58.1630577732 69.0522012554 34 8
55.546605245 58.1630577732 34 9
52.1139340511 55.546605245 34 15
12.9249470849 52.1139340511 50 32
1.69662714199 12.9249470849 50 33
0.363284474801 1.69662714199 51 33
i i j 5

In [ ]:
#finestra de 9x9
millors_pix(finestra,finestra_validar,img_gris_convine,i,j)

In [ ]:
for i in xrange(5,9):
    for j in xrange(5,9):
        print i,j


In [46]:
plt.imshow(img_gris)
plt.title('img_mask')
plt.show()

In [8]:
np.max(img_gris)

256.0

In [ ]:
#codigos de pruevas

img_mask = convine_mask(zona_delete,img_gris)
finestra=5
print np.min(zona_delete)
cont = 0
for i in xrange(len(img_mask)):
    for j in xrange(len(img_mask[0])):
        if img_mask[i][j]==-100:
            #faltaria fer que compari el valor del gradient i el valor del pixel
            t,r= buscar_pix(img_mask,i,j-1,finestra)

            img_mask[i][j]=img_mask[t][r]
            #img_mask[i-5:i+5,j-5:j+5]=part
            #img_mask[i-5:i+5][j-5:j+5]=img_mask[t-5:j+5][r-5:j+5]
            #misc.imsave('imatges/'+str(cont+1)+'.png', img_mask)
            #cont  += 1




plt.imshow(img_mask)
plt.title('img_mask')
plt.show()


print len(img_gris[0])


def bucarpath(img,finestra,t,r):
    path = np.zeros((finestra*2,finestra*2))
    path = img[t-finestra:t+finestra,r-finestra:r+finestra]
    return path
prova=bucarpath(img_gris,5,275,439)
fig = plt.figure(123) 
plt.imshow(img_gris,cmap="gray")
plt.title('img_mask')

sub=bucarpath(img_gris,5,238,413)
img_gris[275-5:275+5,429-5:429+5]=sub
fig = plt.figure(124) 
plt.imshow(img_gris,cmap="gray")
plt.title('img_mask')
plt.show()